In [ ]:
import pandas as pd
import pyodbc 
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy as sal
from sqlalchemy import create_engine

In [ ]:
#Connect to Data Structure for sqlalchemy is: "mssql+pyodbc://OpR-Marc-DB2:1433/RPC1?driver=ODBC+Driver+17+for+SQL+Server" --change RPC1 for other volumes

RPC1 = sal.create_engine("mssql+pyodbc://OpR-Marc-DB2:1433/RPC1?driver=ODBC+Driver+17+for+SQL+Server")
connection = RPC1.connect()

RC1 = sal.create_engine("mssql+pyodbc://OpR-Marc-DB2:1433/RC1?driver=ODBC+Driver+17+for+SQL+Server")
connection = RC1.connect()


In [ ]:
RPC1RibbonInputDF = pd.DataFrame()
RC1RibbonInputDF = pd.DataFrame()

In [ ]:
RC1Aiis= ["514", "3679", "2610", "410"]
RPC1Aiis = ["192", "262", "265", "1685", "2710"]

In [ ]:
#Pull Data from RPC1
for x in RPC1Aiis: 
    query = "SET NOCOUNT ON; EXEC RLPfeiffer.RibbonInputs @TargetCell= " + "'" + x + "'"
    RI = pd.read_sql(query, con = RPC1)
    RI['Volume']= 'RPC1'
    RPC1RibbonInputDF = RPC1RibbonInputDF.append(RI)

RPC1RibbonInputDF

In [ ]:
#Fix any odd names in RC1 DF or simplify *This is currently a manual step)
RPC1RibbonInputDF.replace('CBa?', "CBa", inplace=True)
RPC1RibbonInputDF.replace('CBbwf', "CBb", inplace=True)

In [ ]:
RPC1RibbonInputDF.SourceParentLabel.value_counts()

In [ ]:
RPC1RibbonInputDF.TargetParent.value_counts()

In [ ]:
RPC1RibbonInputDF.groupby(["TargetParent"])["SourceParentLabel"].value_counts()

In [ ]:
RPC1RibbonInputDF = RPC1RibbonInputDF.reset_index(drop = True)

In [ ]:
RPC1RibbonInputDF[["SourceParentLabel", "PSDArea"]].groupby("SourceParentLabel").describe()

In [ ]:
#Pull data from RC1
for x in RC1Aiis: 
    query = "SET NOCOUNT ON; EXEC RLPfeiffer.RibbonInputs @TargetCell= " + "'" + x + "'"
    RI = pd.read_sql(query, con = RC1)
    RI['Volume']= 'RC1'
    RC1RibbonInputDF = RC1RibbonInputDF.append(RI)

RC1RibbonInputDF

In [ ]:
#Fix all expanded names in RC! (subclasses of BCs beyond a and b are excluded in this analysis)
RC1RibbonInputDF.replace('CBa?', "CBa", inplace=True)
RC1RibbonInputDF.replace('CBa1', "CBa", inplace=True)
RC1RibbonInputDF.replace('CBa2', "CBa", inplace=True)
RC1RibbonInputDF.replace('CBa1-2', "CBa", inplace=True)
RC1RibbonInputDF.replace('CBa2-3', "CBa", inplace=True)
RC1RibbonInputDF.replace('CBa2-4', "CBa", inplace=True)
RC1RibbonInputDF.replace('CBb6', "CBb", inplace=True)
RC1RibbonInputDF.replace('CBb5', "CBb", inplace=True)
RC1RibbonInputDF.replace('CBbwf', "CBb", inplace=True)

In [ ]:
RC1RibbonInputDF = RC1RibbonInputDF.reset_index(drop = True)

In [ ]:
RC1RibbonInputDF.SourceParentLabel.value_counts()

In [ ]:
RC1RibbonInputDF.TargetParent.value_counts()

In [ ]:
RC1RibbonInputDF.groupby(["TargetParent"])["SourceParentLabel"].value_counts()

In [ ]:
df = RPC1RibbonInputDF
df.loc[df["PSDArea"].idxmax()]

In [ ]:
#concatinate the 2 datasets
frames = [RPC1RibbonInputDF, RC1RibbonInputDF]
CombinedRibbonInputDF = pd.concat(frames).reset_index()

In [ ]:
#Use this if/when you want to export something to an excel file (use this for tableau analysis)
CombinedRibbonInputDF.to_excel(r'D:\Documents\Data\RPC1\Aii_GAC_Project\dfExports\CombinedRibbonInput.xlsx')

In [ ]:

CombinedRibbonInputDF.groupby("TargetParent")['SourceParent'].value_counts()

In [ ]:
CombinedRibbonInputDF

In [ ]:

# Count the number of unique Source Parents for each Label using value_counts
unique_source_parents = CombinedRibbonInputDF.groupby(['Volume', 'TargetParent', 'SourceParentLabel'])['SourceParent'].nunique().to_frame(name = 'InputCells').reset_index()

# Print the result
print( unique_source_parents)


In [ ]:
SimpRibbonInputDF = CombinedRibbonInputDF.groupby(['Volume', 'TargetParent'])["SourceParentLabel"].value_counts().to_frame(name = 'RibbonCount').reset_index()

In [ ]:
df = SimpRibbonInputDF
df["TotalRibbons"] = df.groupby('TargetParent')['RibbonCount'].transform('sum')
df["PercentofInput"] = df["RibbonCount"] / df["TotalRibbons"]*100

print(SimpRibbonInputDF)

In [ ]:
mean_std_df = SimpRibbonInputDF.groupby(['Volume', 'SourceParentLabel'])['PercentofInput'].agg(['mean', 'std'])
print(mean_std_df)

In [ ]:
CombinedRibbonInputDF.query("TargetParent==265")["SourceParent"].value_counts()